<a href="https://colab.research.google.com/github/Reptilefury/coursera-machine-learning/blob/main/Building_Image_Pair_generators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementing image pair generation for siamese neural networks

In [ ]:
from tensorflow.keras.datasets import mnist
from imutils import build_montages
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
def make_pairs(images,labels): #This method takes two classes images and labels the images 
  #Two empty lists to hold the images pairs (image,image) and labels to indicate if a pair is positive or negative 
  pair_images = []
  pair_labels = []
  #Calculate the total number of unque classes in our dataset
  num_classes = len(np.unique(labels))
  idx = [np.where(labels == i)[0] for i in range(0, num_classes)]
  #We grab the image and label belonging to the current iteration
  for idxA in range(len(images)):
   current_image = images[idxA]
   label = labels[idxA]  
   #We randomly pick an image that belongs to the same class and label 
   idxB = np.random.choice(idx[label])
   pos_image = images[idxB]
   #Prepare the positive pair and update the images and labels 
   #lists respectively 
   pair_images.append([current_image,pos_image])
   pair_labels.append([1])
   #Grab the indices for each class labels *not* equal to
   #the current label and randomly pick an image corresponding to the current label 
   negIdx = np.where(labels != label)[0]
   negImage = images[np.random.choice(negIdx)]
   #Prepare a negative pair of indices and update our lists
   pair_images.append([current_image,negImage])
   pair_labels.append([0])
   #Return a two tuple of our images pairs and labels 
  return (np.array(pair_images),np.array(pair_labels))

In [ ]:
#Load the mnist dataset and scale the pixel values to the range of 0 and 1(normalize/standardize)
(train_images, train_labels),(test_images,test_labels)  = mnist.load_data()

In [ ]:
#train_images = train_images.astype('float32')/255.0

In [ ]:
(pair_train,label_train) = make_pairs(train_images,train_labels)

In [ ]:
pair_train[0].shape #Get the first images in the first tuple  with is of size 2 ,28 * 28 this means 2 images of size 28 * 28 

(2, 28, 28)

In [ ]:
#Make the pairs for the test set
(pairs_test,test_labels) = make_pairs(test_images,test_labels)

In [ ]:
#Initialize the list that will be used when building our montage 
images = []

In [ ]:
#Loop all over the sample our training pairs
for i in np.random.choice(np.arange(0, len(pair_train)), size =(49,)):
  #Grab the current image pair and label
  imageA = pair_train[i][0]
  imageB = pair_train[i][1]
  label = label_train[i]
  #To make it easier to visualize the pair and their positive or negative annotations 
  #we are going to pad the pair along with four pixels  on the top, bottom and right borders respectively
  output = np.zeros((36,60),dtype="uint8")
  pair = np.hstack([imageA,imageB])
  output[4:32,0:56] = pair 

  #Set the next text label for the pair along with what color we are going 
  #to draw the pair in (green for a "positive pair" and "red for a negative pair")
  
  text =  "neg" if label[0] == 0 else "pos"
  color = (0,0,255) if label[0] == 0 else (0 ,255 , 0) 
  #Create a 3 channel RGB image from grayscale pair
  #resize it from 60 x 36 to 96 by 51 so we can better see it and then draw what type of image pair is
  #on the image 
  vis = cv2.merge([output] * 3)
  vis = cv2.resize(vis,(96,51), interpolation = cv2.INTER_LINEAR)

  cv2.putText(vis,text,(2,12), cv2.FONT_HERSHEY_SIMPLEX, 0.75, color,2)
  #Add the pair visualization to our list of output images
  images.append(vis)
  #Construct the montage for our image and put it to out screen 
  montage = build_montages(images,(96,51),(7,7))[0]
  #Show the output of the montage 
  #cv2.imshow("Siamese Image Pairs", montage)
  plt.figure(figsize = (10,10))

  plt.imshow(montage)
  cv2.waitKey(0)

Given the index list we must start generating our positive and negative pairs. 